In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_from_disk

from config import lima_filtered_paraphrased_dataset_path, hf_model_id
import torch
import numpy as np
import random
import gc

from utilities.preprocessing import prepare_dataset
from utilities.gradient_operations import get_gradients, get_flattened_weight_vector
from open_instruct.dataset_processor import SFTDatasetProcessor, DatasetConfig, CHAT_TEMPLATES

In [2]:
# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
torch.cuda.empty_cache()
gc.collect()

60

In [4]:
model = AutoModelForCausalLM.from_pretrained(hf_model_id)
tokenizer = AutoTokenizer.from_pretrained(hf_model_id, return_tensors="pt")

print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)

50279
1


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
model.eval()

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): OlmoRotaryEmbedding()
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
  )
  (

In [7]:
dataset = load_from_disk(lima_filtered_paraphrased_dataset_path)
dataset

Dataset({
    features: ['id', 'messages', 'paraphrased_messages'],
    num_rows: 988
})

In [8]:
dataset["messages"][0][0]["content"]

'Can brain cells move? By movement I mean long distance migration (preferably within the brain only).'

In [9]:
dataset["messages"][0][1]["content"]

'The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and Miller, 2002).\nNeuronal stem cells migrate over long distances in response to injury (Imitola et al., 2004) and they migrate from specific stem-cell locations (e.g.

In [21]:
dataset_config = DatasetConfig(
    chat_template="tulu", 
    max_token_length=model.config.max_position_embeddings,
    train_only_on_prompt=True
)
dataset_config

DatasetConfig(chat_template='tulu', preference_chosen_key='chosen', preference_rejected_key='rejected', sft_messages_key='messages', binary_messages_key='messages', label='binary_labels', convert_preference_to_binary_dataset=False, max_token_length=2048, max_prompt_token_lenth=None, sanity_check=False, sanity_check_max_samples=100, batched=False, load_from_cache_file=True, num_proc=12, train_only_on_prompt=True, ncols=2)

In [22]:
train_dataloader = prepare_dataset(dataset=dataset, dataset_config=dataset_config, tokenizer=tokenizer)

In [48]:
dataset

Dataset({
    features: ['id', 'messages', 'paraphrased_messages'],
    num_rows: 988
})

In [12]:
train_dataloader

Dataset({
    features: ['id', 'messages', 'paraphrased_messages', 'input_ids_prompt', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 934
})

In [20]:
print(len(train_dataloader))

988


In [15]:
sample_0 = train_dataloader[0]
sample_0

{'id': 'lima_0',
 'messages': [{'role': 'user',
   'content': 'Can brain cells move? By movement I mean long distance migration (preferably within the brain only).'},
  {'role': 'assistant',
   'content': 'The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form

In [14]:
gradients_sample_0 = get_gradients(model, sample_0)

NameError: name 'sample_0' is not defined

In [21]:
for key, value in gradients_sample_0.items():
    print(f"{key}: {value.shape}")

model.embed_tokens.weight: torch.Size([50304, 2048])
model.layers.0.self_attn.q_proj.weight: torch.Size([2048, 2048])
model.layers.0.self_attn.k_proj.weight: torch.Size([2048, 2048])
model.layers.0.self_attn.v_proj.weight: torch.Size([2048, 2048])
model.layers.0.self_attn.o_proj.weight: torch.Size([2048, 2048])
model.layers.0.mlp.gate_proj.weight: torch.Size([8192, 2048])
model.layers.0.mlp.up_proj.weight: torch.Size([8192, 2048])
model.layers.0.mlp.down_proj.weight: torch.Size([2048, 8192])
model.layers.1.self_attn.q_proj.weight: torch.Size([2048, 2048])
model.layers.1.self_attn.k_proj.weight: torch.Size([2048, 2048])
model.layers.1.self_attn.v_proj.weight: torch.Size([2048, 2048])
model.layers.1.self_attn.o_proj.weight: torch.Size([2048, 2048])
model.layers.1.mlp.gate_proj.weight: torch.Size([8192, 2048])
model.layers.1.mlp.up_proj.weight: torch.Size([8192, 2048])
model.layers.1.mlp.down_proj.weight: torch.Size([2048, 8192])
model.layers.2.self_attn.q_proj.weight: torch.Size([2048, 2

In [23]:
decoded_input = tokenizer.decode(token_ids=sample_0["input_ids"])
decoded_input

'<|user|>\nCan brain cells move? By movement I mean long distance migration (preferably within the brain only).\n<|assistant|>\nThe question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and Miller, 2002).\nNeuronal stem cells mi

In [16]:
decoded_output = tokenizer.decode(token_ids=[(tokenizer.pad_token_id if token == -100 else token) for token in sample_0["labels"]])
decoded_output

'<|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|>The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of mig

In [26]:
dataset_processor = SFTDatasetProcessor(tokenizer, dataset_config)

In [28]:
tokenized_dataset = dataset_processor.tokenize(dataset)

In [29]:
tokenized_dataset[0]

{'id': 'lima_0',
 'messages': [{'role': 'user',
   'content': 'Can brain cells move? By movement I mean long distance migration (preferably within the brain only).'},
  {'role': 'assistant',
   'content': 'The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form

In [30]:
len(tokenized_dataset[0]["input_ids"])

363

In [31]:
len(sample_0["input_ids"][0])

TypeError: object of type 'int' has no len()

In [33]:
decoded_output = tokenizer.decode(token_ids=[(tokenizer.pad_token_id if token == -100 else token) for token in sample_0["labels"]])
decoded_output

'<|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|>The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.\nHowever, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.\nIn  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of mig